# Experiments 1-3
- **Model:** `yolov8n` *(Nano)*
- **Dataset:** 3.5m | 90º
- **Sizes:** small & mid
- **Experiments:**
    1. small + mid
    2. small
    3. mid


## Init

In [9]:
import os
import shutil
import fnmatch

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

## Helper Functions

In [ ]:
# Change for different file formats
reference = {
  "small": {
    "suffix": ".S",
    "file": "209"
  },
  "mid": {
    "suffix": ".M",
    "file": "503"
  },
  "large": {
    "suffix": ".L",
    "file": "000"
  }
}

In [ ]:
# Clone config files
def copy_config(src_folder, dest_folder):
    """
    Copies files from src_folder to dest_folder, excluding subfolders.

    Args:
        src_folder: The path to the source folder.
        dest_folder: The path to the destination folder.
    """

    try:
        # Ensure destination folder exists
        os.makedirs(dest_folder, exist_ok=True)

        for filename in os.listdir(src_folder):
            src_path = os.path.join(src_folder, filename)
            dest_path = os.path.join(dest_folder, filename)

            if os.path.isfile(src_path):
                shutil.copy2(src_path, dest_path) #copy metadata as well.
                #Use shutil.copy for not copying metadata.
                print(f"Copied: {filename}")
            #else: #optional
                #print(f"Skipped (not a file): {filename}") #optional. Uncomment if you want to see the skipped folders.

        print("✅ Copying complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")


In [ ]:
# Copy filtered dataset images/labels
def copy_and_filter_folder(src_folder, dest_folder, pattern):
    """
    Copies a folder and files that match the given pattern.
    Alerts the user when a folder or file already exists but *does not* overwrite.
    Creates only what is needed.

    :param src_folder: Path to the source folder.
    :param dest_folder: Path to the destination folder.
    :param pattern: Filename pattern to keep (e.g., "*.txt").
    """
    try:
        # Ensure destination folder exists
        if not os.path.exists(dest_folder):
            print(f"✓ Creating destination folder '{dest_folder}'.\n")
            os.makedirs(dest_folder)
        else:
            print(f"✓ Destination folder '{dest_folder}' already exists.\n")

        # Walk through the source folder
        for root, _, files in os.walk(src_folder):
            relative_path = os.path.relpath(root, src_folder)
            new_root = os.path.join(dest_folder, relative_path)

            if not os.path.exists(new_root):
                print(f"Creating subdirectory '{new_root}'")
                os.makedirs(new_root)
            else:
                print(f"❕Subdirectory '{new_root}' already exists.")
                print("Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.")

            for file in files:
                if fnmatch.fnmatch(file, pattern + "*"):
                    src_file = os.path.join(root, file)
                    dest_file = os.path.join(new_root, file)

                    if not os.path.exists(dest_file):
                        shutil.copy2(src_file, dest_file)  # copy metadata as well
                    else:
                        print(f"❗️File '{dest_file}' already exists. Skipping.")

            print(f" ✓ Copying files complete.\n")
        print("✅ Copying dataset complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")

In [ ]:
def save_on_cloud(source: str, destination: str):
    """
    Saves a folder to a cloud storage location (e.g., Google Drive in Colab).

    Args:
        source (str): The path to the source folder.
        destination (str): The path to the destination folder (in cloud storage).
    """
    # 0. Input Validation (Assertions)
    assert isinstance(source, str), "Source must be a string."
    assert isinstance(destination, str), "Destination must be a string."
    
    try:
        # 1. Verify Source Folder
        if not os.path.exists(destination):
            os.makedirs(destination)

        # 2. Copy the Folder
        shutil.copytree(source, destination, dirs_exist_ok=True)
        print("✅ Folder copied successfully:\n  ",source,"\n  -->",destination)

    except Exception as e:
        print(f"❌ An error occurred: {e}")

# Datasets builder

## Importing from Drive

In [6]:
!rm -rf /content/sample_data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Check if the cloud path is ok and the dataset can be found
!ls /content/drive/MyDrive/YOLO

3.5m.v3i.yolov8


In [9]:
drive_path = '/content/drive/MyDrive/YOLO'
drive_datasets_paths = os.listdir(drive_path)
drive_datasets = len(drive_datasets_paths)
if (drive_datasets) > 1:
    print("There are %d dataset options:" % drive_datasets)
else:
    print("Theres is only 1 dataset:")
drive_datasets_paths

Theres is only 1 dataset:


['3.5m.v3i.yolov8']

In [10]:
choose_dataset = 1
index = choose_dataset - 1
model = os.listdir(drive_path)[index]
print("Choosed model:", model)

Choosed model: 3.5m.v3i.yolov8


***Readme:***
*   **Option 1:** is desirable if you need to test many subset combinations in the same session (avoid downloading data twice from the cloud).
*   **Option 3:** is best if you're just going to test one subset combination  (avoid downloading unnecessary data from the cloud).



In [11]:
# Option 1 (download full dataset)
!cp -r /content/drive/MyDrive/YOLO/ /content
src_folder = f"/content/YOLO/{model}"

In [12]:
# Option 3 (download just what's needed)
# src_folder = f"{drive_path}/{model}"

## Build dataset 'small'

### Settings

In [13]:
# CHOOSE THE DATASET YOU WANT TO WORK WITH
choosen_data = 'small' # Options: small / mid / large

In [14]:
suffix = reference[choosen_data]['suffix']
dest_folder = f"/content/YOLO/{model}{suffix}"
pattern = reference[choosen_data]['file']

In [15]:
print(f'🆗 SETTING PARAMETERS\n - Model: {model}\n - rigin: {src_folder}\n - Destination: {dest_folder}\n - Prefix: {pattern}…')

🆗 SETTING PARAMETERS
 - Model: 3.5m.v3i.yolov8
 - rigin: /content/YOLO/3.5m.v3i.yolov8
 - Destination: /content/YOLO/3.5m.v3i.yolov8.S
 - Prefix: 209…


### Builder

In [16]:
copy_config(src_folder, dest_folder)

Copied: README.roboflow.txt
Copied: README.dataset.txt
Copied: .DS_Store
Copied: data.yaml
✅ Copying complete.


In [17]:
copy_and_filter_folder(src_folder, dest_folder, pattern)

✓ Destination folder '/content/YOLO/3.5m.v3i.yolov8.S' already exists.

❕Subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/.' already exists.
Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/train'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/train/images'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/train/labels'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/valid'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/valid/images'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.S/valid/labels'
 ✓ Copying files complete.

✅ Copying dataset complete.


## Build dataset 'medium'

### Settings

In [18]:
# CHOOSE THE DATASET YOU WANT TO WORK WITH
choosen_data = 'mid' # Options: small / mid / large

In [19]:
suffix = reference[choosen_data]['suffix']
dest_folder = f"/content/YOLO/{model}{suffix}"
pattern = reference[choosen_data]['file']

In [20]:
print(f'🆗 SETTING PARAMETERS\n - Model: {model}\n - rigin: {src_folder}\n - Destination: {dest_folder}\n - Prefix: {pattern}…')

🆗 SETTING PARAMETERS
 - Model: 3.5m.v3i.yolov8
 - rigin: /content/YOLO/3.5m.v3i.yolov8
 - Destination: /content/YOLO/3.5m.v3i.yolov8.M
 - Prefix: 503…


### Builder

In [21]:
copy_config(src_folder, dest_folder)

Copied: README.roboflow.txt
Copied: README.dataset.txt
Copied: .DS_Store
Copied: data.yaml
✅ Copying complete.


In [22]:
copy_and_filter_folder(src_folder, dest_folder, pattern)

✓ Destination folder '/content/YOLO/3.5m.v3i.yolov8.M' already exists.

❕Subdirectory '/content/YOLO/3.5m.v3i.yolov8.M/.' already exists.
Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.M/train'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.M/train/images'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.M/train/labels'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.M/valid'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.M/valid/images'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3.5m.v3i.yolov8.M/valid/labels'
 ✓ Copying files complete.

✅ Copying dataset complete.


## Remove datasets (restart)

In [23]:
# Removes complete local dataset
#!rm -rf /content/YOLO

# Model constructor
Model: `yolov8n` *(Nano)*



## Download model

In [24]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolov8n.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 104MB/s]


# Finetuning

### Info

In [25]:
!nvidia-smi

Tue Feb 11 23:33:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [26]:
!yolo version

8.3.74


-----
## Experiment 1 *(full dataset)*
### *3.5m | 90º | small + mid*

### Train

In [27]:

# Train the model
model.train(
    data="/content/YOLO/3.5m.v3i.yolov8/data.yaml",
    epochs=100,
    imgsz=240,
    freeze=10,
    batch=64
)


Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO/3.5m.v3i.yolov8/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=240, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 21.7MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 99.0MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[240] must be multiple of max stride 32, updating to [256]


...

                   all        736       4084      0.455      0.425      0.392       0.13



100 epochs completed in 0.299 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:05<00:00,  1.12it/s]


                   all        736       4084      0.458      0.427      0.395      0.132
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ea9aa64d950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [28]:
# Load the trained YOLO model
model1 = YOLO("/content/runs/detect/train/weights/best.pt")

# Validate the model
model1.val(data="/content/YOLO/3.5m.v3i.yolov8/data.yaml")

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/YOLO/3.5m.v3i.yolov8/valid/labels.cache... 736 images, 28 backgrounds, 0 corrupt: 100%|██████████| 736/736 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:05<00:00,  9.09it/s]


                   all        736       4084       0.46      0.428      0.395      0.132
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ea9ab333350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Inference

#### Test 1 (small weed)

In [35]:
image = '/content/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [30]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project="model1(3.5_SM)", name="experiment1")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile00x00.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 256x256 5 plant-weeds, 14.6ms
Speed: 1.4ms preprocess, 14.6ms inference, 4.1ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model1(3.5_SM)/experiment1
Clase: 0, Confianza: 0.50, BBox: (70.88, 109.10, 129.07, 135.78)
Clase: 0, Confianza: 0.41, BBox: (82.53, 95.12, 160.95, 131.97)
Clase: 0, Confianza: 0.29, BBox: (68.26, 139.13, 106.90, 148.60)
Clase: 0, Confianza: 0.28, BBox: (74.13, 99.27, 129.02, 135.36)
Clase: 0, Confianza: 0.27, BBox: (186.95, 156.66, 239.85, 173.68)


In [31]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.17083333333333334 0.44166666666666665 0.14166666666666666 0.08333333333333333
0 0.22083333333333333 0.49583333333333335 0.2 0.058333333333333334
0 0.24166666666666667 0.5166666666666667 0.06666666666666667 0.075
0 0.5041666666666667 0.4791666666666667 0.25833333333333336 0.13333333333333333
0 0.39166666666666666 0.475 0.23333333333333334 0.13333333333333333
0 0.9166666666666666 0.6958333333333333 0.16666666666666666 0.041666666666666664



#### Test 2 (medium weed)

In [36]:
image = '/content/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [ ]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project="model3(3.5_SM)", name="experiment2")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile00x00.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 256x256 5 plant-weeds, 11.6ms
Speed: 1.5ms preprocess, 11.6ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model1(3.5_SM)/experiment2
Clase: 0, Confianza: 0.50, BBox: (70.88, 109.10, 129.07, 135.78)
Clase: 0, Confianza: 0.41, BBox: (82.53, 95.12, 160.95, 131.97)
Clase: 0, Confianza: 0.29, BBox: (68.26, 139.13, 106.90, 148.60)
Clase: 0, Confianza: 0.28, BBox: (74.13, 99.27, 129.02, 135.36)
Clase: 0, Confianza: 0.27, BBox: (186.95, 156.66, 239.85, 173.68)


In [34]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.17083333333333334 0.44166666666666665 0.14166666666666666 0.08333333333333333
0 0.22083333333333333 0.49583333333333335 0.2 0.058333333333333334
0 0.24166666666666667 0.5166666666666667 0.06666666666666667 0.075
0 0.5041666666666667 0.4791666666666667 0.25833333333333336 0.13333333333333333
0 0.39166666666666666 0.475 0.23333333333333334 0.13333333333333333
0 0.9166666666666666 0.6958333333333333 0.16666666666666666 0.041666666666666664



### Save results

In [ ]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save/


In [ ]:
# Store experiments
save_on_cloud(source='/content/model1(3.5_SM)/', destination='/content/drive/MyDrive/save.SM/')

✅ Folder copied successfully:
   /content/model1(3.5_SM)/ 
  --> /content/drive/MyDrive/save.SM/


----------
## Experiment 2 *(small weed)*
### *3.5m | 90º | small*

### Train

In [49]:

# Train the model
model.train(
    data="/content/YOLO/3.5m.v3i.yolov8.S/data.yaml",
    epochs=100,
    imgsz=240,
    freeze=10,
    batch=64
)


Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO/3.5m.v3i.yolov8.S/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=240, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

train: Scanning /content/YOLO/3.5m.v3i.yolov8.S/train/labels... 1104 images, 165 backgrounds, 0 corrupt: 100%|██████████| 1104/1104 [00:00<00:00, 2231.19it/s]


train: New cache created: /content/YOLO/3.5m.v3i.yolov8.S/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/YOLO/3.5m.v3i.yolov8.S/valid/labels... 368 images, 19 backgrounds, 0 corrupt: 100%|██████████| 368/368 [00:00<00:00, 915.13it/s]

val: New cache created: /content/YOLO/3.5m.v3i.yolov8.S/valid/labels.cache


...

                   all        368       1653      0.505      0.442      0.422      0.132



100 epochs completed in 0.205 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.14s/it]


                   all        368       1653      0.505      0.441      0.428      0.135
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ea99210ba50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [53]:
# Load the trained YOLO model
model2 = YOLO("/content/runs/detect/train2/weights/best.pt")

# Validate the model
model2.val(data="/content/YOLO/3.5m.v3i.yolov8.S/data.yaml")

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/YOLO/3.5m.v3i.yolov8.S/valid/labels.cache... 368 images, 19 backgrounds, 0 corrupt: 100%|██████████| 368/368 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:03<00:00,  7.47it/s]


                   all        368       1653      0.504      0.441      0.428      0.136
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ea9acfd3cd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Inference

#### Test 1 (small weed)

In [54]:
image = '/content/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [ ]:
# Ejecutar predicción sobre la imagen
results = model2(image, save=True, project="model2(3.5_S)", name="experiment1")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 256x256 6 plant-weeds, 5.8ms
Speed: 0.8ms preprocess, 5.8ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model1(3.5_S)/experiment1
Clase: 0, Confianza: 0.55, BBox: (53.78, 150.11, 98.77, 167.39)
Clase: 0, Confianza: 0.43, BBox: (0.00, 161.76, 37.01, 179.84)
Clase: 0, Confianza: 0.42, BBox: (45.52, 171.32, 129.15, 209.77)
Clase: 0, Confianza: 0.37, BBox: (176.80, 182.89, 238.33, 218.89)
Clase: 0, Confianza: 0.36, BBox: (180.69, 187.03, 240.00, 232.01)
Clase: 0, Confianza: 0.29, BBox: (138.41, 199.29, 198.97, 213.28)


In [56]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.875 0.85 0.25416666666666665 0.125
0 0.9041666666666667 0.8875 0.19583333333333333 0.13333333333333333
0 0.9625 0.9458333333333333 0.07916666666666666 0.08333333333333333
0 0.7083333333333334 0.8791666666666667 0.30833333333333335 0.06666666666666667
0 0.4375 0.8458333333333333 0.20833333333333334 0.041666666666666664
0 0.4041666666666667 0.825 0.225 0.041666666666666664
0 0.3333333333333333 0.7583333333333333 0.21666666666666667 0.09166666666666666
0 0.2375 0.7541666666666667 0.225 0.041666666666666664
0 0.09583333333333334 0.7291666666666666 0.19166666666666668 0.09166666666666666



#### Test 2 (medium weed)

In [61]:
image = '/content/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [ ]:
# Ejecutar predicción sobre la imagen
results = model2(image, save=True, project="model2(3.5_S)", name="experiment2")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg: 256x256 6 plant-weeds, 18.6ms
Speed: 1.4ms preprocess, 18.6ms inference, 1.9ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model1(3.5_S)/experiment2
Clase: 0, Confianza: 0.54, BBox: (6.13, 0.04, 58.86, 47.09)
Clase: 0, Confianza: 0.45, BBox: (0.43, 199.90, 25.25, 240.00)
Clase: 0, Confianza: 0.38, BBox: (94.58, 163.64, 107.85, 182.44)
Clase: 0, Confianza: 0.37, BBox: (29.29, 133.65, 98.96, 143.72)
Clase: 0, Confianza: 0.33, BBox: (26.18, 4.53, 58.16, 49.48)
Clase: 0, Confianza: 0.25, BBox: (3.89, 126.54, 24.79, 180.22)


In [63]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.05416666666666667 0.7958333333333333 0.10416666666666667 0.13333333333333333
0 0.0875 0.6625 0.175 0.275
0 0.25 0.575 0.5 0.125
0 0.09166666666666666 0.5583333333333333 0.10833333333333334 0.35833333333333334
0 0.05416666666666667 0.48333333333333334 0.10416666666666667 0.23333333333333334
0 0.9875 0.25416666666666665 0.025 0.175
0 0.029166666666666667 0.1375 0.058333333333333334 0.275
0 0.1375 0.10416666666666667 0.21666666666666667 0.20833333333333334
0 0.15416666666666667 0.175 0.20833333333333334 0.275



### Save results

In [ ]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save2/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save2/


In [ ]:
# Store experiments
save_on_cloud(source='/content/model2(3.5_S)/', destination='/content/drive/MyDrive/save.S/')

✅ Folder copied successfully:
   /content/model2(3.5_S)/ 
  --> /content/drive/MyDrive/save.S/


----
## Experiment 3 *(medium weed)*
### *3.5m | 90º | mid*

### Train

In [65]:

# Train the model
model.train(
    data="/content/YOLO/3.5m.v3i.yolov8.M/data.yaml",
    epochs=100,
    imgsz=240,
    freeze=10,
    batch=64
)


Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO/3.5m.v3i.yolov8.M/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=240, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning /content/YOLO/3.5m.v3i.yolov8.M/train/labels... 368 images, 7 backgrounds, 0 corrupt: 100%|██████████| 368/368 [00:00<00:00, 1760.44it/s]

train: New cache created: /content/YOLO/3.5m.v3i.yolov8.M/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/YOLO/3.5m.v3i.yolov8.M/valid/labels... 368 images, 9 backgrounds, 0 corrupt: 100%|██████████| 368/368 [00:00<00:00, 959.11it/s]


val: New cache created: /content/YOLO/3.5m.v3i.yolov8.M/valid/labels.cache
Plotting labels to runs/detect/train22/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 2 dataloader workers
Logging results to runs/detect/train22
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        368       2431      0.427      0.439      0.384      0.125



100 epochs completed in 0.107 hours.
Optimizer stripped from runs/detect/train22/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train22/weights/best.pt, 6.2MB

Validating runs/detect/train22/weights/best.pt...
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.07s/it]


                   all        368       2431      0.443      0.421      0.393      0.137
Speed: 0.0ms preprocess, 0.6ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train22


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ea9aa693590>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [66]:
# Load the trained YOLO model
model3 = YOLO("/content/runs/detect/train3/weights/best.pt")

# Validate the model
model3.val(data="/content/YOLO/3.5m.v3i.yolov8.M/data.yaml")

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/YOLO/3.5m.v3i.yolov8.M/valid/labels.cache... 368 images, 9 backgrounds, 0 corrupt: 100%|██████████| 368/368 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:03<00:00,  6.51it/s]


                   all        368       2431      0.444      0.423      0.395      0.137
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/val3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ea9ab1f1110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Inference

#### Test 1 (small weed)

In [67]:
image = '/content/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [ ]:
# Ejecutar predicción sobre la imagen
results = model3(image, save=True, project="model3(3.5_M)", name="experiment1")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 256x256 6 plant-weeds, 6.4ms
Speed: 1.1ms preprocess, 6.4ms inference, 1.4ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model1(3.5_M)/experiment1
Clase: 0, Confianza: 0.66, BBox: (43.49, 168.38, 130.86, 209.65)
Clase: 0, Confianza: 0.49, BBox: (52.37, 149.49, 98.51, 167.38)
Clase: 0, Confianza: 0.48, BBox: (180.75, 182.76, 240.00, 227.02)
Clase: 0, Confianza: 0.38, BBox: (0.00, 151.12, 41.26, 182.75)
Clase: 0, Confianza: 0.28, BBox: (0.00, 159.23, 40.64, 181.16)
Clase: 0, Confianza: 0.28, BBox: (135.93, 194.06, 205.86, 217.45)


In [69]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.875 0.85 0.25416666666666665 0.125
0 0.9041666666666667 0.8875 0.19583333333333333 0.13333333333333333
0 0.9625 0.9458333333333333 0.07916666666666666 0.08333333333333333
0 0.7083333333333334 0.8791666666666667 0.30833333333333335 0.06666666666666667
0 0.4375 0.8458333333333333 0.20833333333333334 0.041666666666666664
0 0.4041666666666667 0.825 0.225 0.041666666666666664
0 0.3333333333333333 0.7583333333333333 0.21666666666666667 0.09166666666666666
0 0.2375 0.7541666666666667 0.225 0.041666666666666664
0 0.09583333333333334 0.7291666666666666 0.19166666666666668 0.09166666666666666



#### Test 2 (medium weed)

In [70]:
image = '/content/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [ ]:
# Ejecutar predicción sobre la imagen
results = model3(image, save=True, project="model3(3.5_M)", name="experiment2")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg: 256x256 9 plant-weeds, 6.7ms
Speed: 0.7ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model1(3.5_M)/experiment2
Clase: 0, Confianza: 0.46, BBox: (24.55, 0.00, 73.82, 47.93)
Clase: 0, Confianza: 0.39, BBox: (0.00, 0.01, 51.62, 59.11)
Clase: 0, Confianza: 0.38, BBox: (0.00, 192.13, 28.07, 240.00)
Clase: 0, Confianza: 0.33, BBox: (28.59, 0.01, 80.85, 40.22)
Clase: 0, Confianza: 0.33, BBox: (0.00, 0.00, 27.48, 65.78)
Clase: 0, Confianza: 0.32, BBox: (94.75, 163.66, 107.67, 182.83)
Clase: 0, Confianza: 0.31, BBox: (0.00, 76.52, 34.47, 179.65)
Clase: 0, Confianza: 0.27, BBox: (0.00, 177.09, 30.81, 240.00)
Clase: 0, Confianza: 0.26, BBox: (30.44, 132.18, 123.69, 145.91)


In [72]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.05416666666666667 0.7958333333333333 0.10416666666666667 0.13333333333333333
0 0.0875 0.6625 0.175 0.275
0 0.25 0.575 0.5 0.125
0 0.09166666666666666 0.5583333333333333 0.10833333333333334 0.35833333333333334
0 0.05416666666666667 0.48333333333333334 0.10416666666666667 0.23333333333333334
0 0.9875 0.25416666666666665 0.025 0.175
0 0.029166666666666667 0.1375 0.058333333333333334 0.275
0 0.1375 0.10416666666666667 0.21666666666666667 0.20833333333333334
0 0.15416666666666667 0.175 0.20833333333333334 0.275



### Save results

In [ ]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save3/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save3/


In [ ]:
# Store experiments
save_on_cloud(source='/content/model3(3.5_M)/', destination='/content/drive/MyDrive/save.M/')

✅ Folder copied successfully:
   /content/model3(3.5_M)/ 
  --> /content/drive/MyDrive/save.M/


# Conclusions

## Analysis:

### Experiment 1: _small & medium plant sizes_

- mAP50-95(B): Achieved the highest score of 0.2264.
- Precision: Moderate, around 0.33.
- Recall: High, around 0.40.
- Observations: This experiment demonstrates a good balance between precision and recall, resulting in the best overall performance in terms of mAP50-95(B).

### Experiment 2: _small plant sizes_

- mAP50-95(B): Lowest score among the three experiments.
- Precision: Highest, around 0.45.
- Recall: Lowest, around 0.25.
- Observations: While precision is the highest in this setup, the significantly lower recall indicates that the model struggles to identify a large portion of actual objects. This suggests that with less data, the model becomes more selective, leading to fewer false positives but at the cost of missing out on many actual positives.

### Experiment 3: _medium plant sizes_

- mAP50-95(B): Lower than Experiment 1 but higher than Experiment 2.
- Precision: Lowest, around 0.28.
- Recall: Highest, around 0.50.
- Observations: This experiment prioritizes recall over precision. With the least amount of data, the model casts a wider net, capturing more actual positives but also including more false positives, hence the lower precision.

### Conclusions:
- Dataset Size Matters: The size of the dataset used for fine-tuning plays a crucial role in the model's performance.
- Trade-off between Precision and Recall: There's an evident trade-off between precision and recall. Depending on the application, one might be prioritized over the other. For instance, if it's critical to not miss any actual objects (e.g., in medical diagnosis), high recall would be preferred even at the cost of lower precision.
- Optimal Balance: Using 50% of the dataset seems to provide the best balance between precision and recall for object detection tasks.
- Further Exploration: It would be interesting to experiment with different dataset sizes and observe the changes in the model's performance. Additionally, exploring techniques to mitigate the impact of reduced dataset sizes, such as data augmentation or transfer learning, could be beneficial.